# Pelatihan Model Vehicle Detection

**Tujuan:** Melatih model YOLO berbasis Deep Learning untuk mendeteksi 3 kelas: bus, car, dan van.

## Menghubungkan ke Google Drive

In [6]:
from google.colab import drive

print("Menghubungkan ke Google Drive...")
drive.mount('/content/drive')
print("Google Drive berhasil terhubung di /content/drive")

Menghubungkan ke Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive berhasil terhubung di /content/drive


## Setup dan Ekstrak Dataset

In [12]:
%pip install -q ultralytics supervision pandas

import os
import zipfile
import yaml
from ultralytics import YOLO


ZIP_FILE = "/content/drive/MyDrive/Capstone Project JCAI/Capstone 4/dataset.zip"
# ---------------------------

DATASET_DIR = "vehicle_dataset"

if os.path.exists(ZIP_FILE):
    print(f"Membaca '{ZIP_FILE}' dari Google Drive...")
    print(f"Mengekstrak ke folder ./{DATASET_DIR}...")
    with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
        zip_ref.extractall(DATASET_DIR)
    print("Ekstraksi selesai.")

    print(f"\nMemeriksa isi folder '{DATASET_DIR}':")
    for root, dirs, files in os.walk(DATASET_DIR):
        level = root.replace(DATASET_DIR, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)

        for f in files[:5]:
            print(f'{subindent}{f}')
        if len(files) > 5:
            print(f'{subindent}...')


else:
    print(f"ERROR: File '{ZIP_FILE}' tidak ditemukan di Google Drive.")
    print("Pastikan Anda sudah mengunggah 'dataset.zip' ke Google Drive dan path-nya benar.")

Membaca '/content/drive/MyDrive/Capstone Project JCAI/Capstone 4/dataset.zip' dari Google Drive...
Mengekstrak ke folder ./vehicle_dataset...
Ekstraksi selesai.

Memeriksa isi folder 'vehicle_dataset':
vehicle_dataset/
    vehicle-detection.v1i.yolov12/
        README.dataset.txt
        data.yaml
        README.roboflow.txt
        train/
            labels/
                Screenshot-2025-04-07-182941_png.rf.1cc0f750fce15ab92293e7ff073f7aa3.txt
                image-78-_jpeg_jpg.rf.4b798b4311c13dfb940790293b4cd230.txt
                Screenshot-2025-04-08-110134_png.rf.9f02859111f568994967aa97e52cadfc.txt
                -66F945A5-3856-4F91-9EDD-6EF4833F57FA-png_jpg.rf.dafd08d9cca9cd5e6ded8e10d4f8340e.txt
                Screenshot-2025-04-02-151144_png.rf.a353fa7f073ea2600c174e78a347dced.txt
                ...
            images/
                -5A06964D-6D40-4132-A769-F21A3B8E1DAA-png_jpg.rf.774d2dec838f2999a3579e0a76832691.jpg
                tyoto_minivan1273_png.rf.40b5163ec7b

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Konfigurasi Data Pipeline (data.yaml)

In [16]:
BASE_DATASET_PATH = os.path.join(os.path.abspath(DATASET_DIR), 'vehicle-detection.v1i.yolov12')

data_config = {
    'path': '.', # Set path to current directory or empty to avoid confusion with absolute paths below
    'train': os.path.join(BASE_DATASET_PATH, 'train/images'),
    'val': os.path.join(BASE_DATASET_PATH, 'valid/images'),
    'test': os.path.join(BASE_DATASET_PATH, 'test/images'),
    'nc': 3,
    'names': ['bus', 'car', 'van']
}

# Verification for debugging
print(f"Verifying paths in config.yaml before writing:")
print(f"  Full train path: {data_config['train']} exists: {os.path.exists(data_config['train'])}")
print(f"  Full val path:   {data_config['val']} exists: {os.path.exists(data_config['val'])}")
print(f"  Full test path:  {data_config['test']} exists: {os.path.exists(data_config['test'])}")
# End Verification

CONFIG_FILE = "config.yaml"
with open(CONFIG_FILE, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"File {CONFIG_FILE} berhasil dibuat.")
print("--- Isi config.yaml ---")
with open(CONFIG_FILE, 'r') as f:
    print(f.read())

File config.yaml berhasil dibuat.
--- Isi config.yaml ---
names:
- bus
- car
- van
nc: 3
path: /content/vehicle_dataset/vehicle-detection.v1i.yolov12
test: test/images
train: train/images
val: valid/images



## Pelatihan (Transfer Learning) & Augmentasi


In [19]:
model = YOLO('yolo12n.pt')

# Latih model
results = model.train(
    data=CONFIG_FILE,
    epochs=50,
    imgsz=640,
    patience=10,
    batch=16,
    optimizer='AdamW',

    hsv_h=0.015,  # Perubahan Hue (warna)
    hsv_s=0.7,    # Perubahan Saturation
    hsv_v=0.4,    # Perubahan kecerahan/kontras
    erasing=0.4,  # Simulasi noise

    fliplr=0.0,   # Flip horizontal
    degrees=0.0,  # Rotasi
    mosaic=0.0    # Mosaic
)

print("Pelatihan selesai.")

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=

## Evaluasi Model

In [25]:
import os
from ultralytics import YOLO

MODEL_TERBAIK_PATH = 'runs/detect/train6/weights/best.pt'

if os.path.exists(MODEL_TERBAIK_PATH):
    model_terbaik = YOLO(MODEL_TERBAIK_PATH)

    print("Mengevaluasi model pada test set...")
    metrics = model_terbaik.val(split='test')

    print("--- Metrik Evaluasi Akhir (Test Set) ---")
    print(f"mAP@0.5:0.95 (Metrik Utama): {metrics.box.map * 100:.2f}%")
    print(f"mAP@0.5 (Metrik Sekunder): {metrics.box.map50 * 100:.2f}%")
else:
    print(f"ERROR: Pelatihan gagal atau model '{MODEL_TERBAIK_PATH}' tidak ditemukan.")

Mengevaluasi model pada test set...
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 159 layers, 2,557,313 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1212.2±869.8 MB/s, size: 47.1 KB)
val: Scanning /content/vehicle_dataset/vehicle-detection.v1i.yolov12/test/labels.cache... 220 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 220/220 126.0Kit/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 14, len(boxes) = 413. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 14/14 3.0it/s 4.6s
                   all        220        413      0.869      0.784      0.875      0.683
                   bus         67        109      0.859 

In [28]:
# Simpan Model ke Google Drive
import shutil
import os

PATH_MODEL_ASLI = 'runs/detect/train6/weights/best.pt'

PATH_TUJUAN_GDRIVE = '/content/drive/MyDrive/best_model_intishar.pt'

if os.path.exists(PATH_MODEL_ASLI):
    print(f"Menyalin model terbaik dari {PATH_MODEL_ASLI} ke Google Drive")
    shutil.copy(PATH_MODEL_ASLI, PATH_TUJUAN_GDRIVE)
    print(f"Selesai. Model Anda sekarang aman di: {PATH_TUJUAN_GDRIVE}")
else:
    print(f"ERROR: Tidak dapat menemukan {PATH_MODEL_ASLI}")

Menyalin model terbaik dari runs/detect/train6/weights/best.pt ke Google Drive
Selesai. Model Anda sekarang aman di: /content/drive/MyDrive/best_model_intishar.pt
